# GPU Performance Analysis

This tutorial will explore how to evaluate and optimize the performance of GPU-accelerated codes.
The goal is to:
- Understand the expected optimal performance of your code.
- Identify how far the actual performance deviates from the optimal.
- Explore reasons for performance gaps and practical methods to bridge them.

The material is based on a distilled version of the NHR@FAU workshop GPU Performance Engineering and designed for a two hour event.
Feel free to check out the material on [github](https://github.com/SebastianKuckuk/gpu-performance-engineering) in case you are interested, dates for upcoming public iterations for the European region on the [training overview page](https://hpc.fau.de/teaching/tutorials-and-courses/) of NHR@FAU.

# Performance Models

Performance models aim at relating algorithms, their implementation, hardware characteristics and the expected performance (execution time).
One of the most straight-forward ones is the *bottleneck model*.

## Bottleneck Model

The bottleneck model analyzes potential performance limiters individually, such as:
* **DRAM Bandwidth**, i.e. the speed at which data can be transferred from/ to global memory.
* **Compute Throughput**, i.e. the rate at which computations can be executed.
* **Cache Bandwidth**, i.e. the speed of accessing data from different parts of the memory hierarchy, e.g. L2 cache, L1 cache, or shared memory.

Each bottleneck is modeled separately to estimate its contribution to the total execution time.

In other words: application performance is generally limited by the time required to do the meaningful computations, or by getting data to and from where the compute is happening.

# Our Test Application - 2D Stencil Updates

## CPU Baseline

We begin with a simple yet widely used benchmark: a 2D stencil application.
It can be regarded as a proxy application for (matrix-free) matrix-vector multiplications, which are ubiquitous in HPC applications.

A serial baseline *CPU-only* implementation can be found in [stencil-2d-base.cpp](../src/stencil-2d/stencil-2d-base.cpp).
Reviewing the implementation reveals these key points:
* The main workload is encapsulated in the `stencil2d` function.
* The application can be parameterized with command line arguments (c.f. `parseCLA_2d` in [stencil-2d-util.h](../src/stencil-2d/stencil-2d-util.h)):
  - **Data type**: `float` or `double`
  - **nx, ny**: Grid dimensions, defining total workload (`nx * ny`)
  - **nWarmUp**: Number of non-timed warm-up iterations
  - **nIt**: Number of timed iterations
* Basic diagnostic output of performance data is available via the `printStats` function in [util.h](../src/util.h).
  - Key **performance metrics** such as the sustained bandwidth are *estimated*

After reviewing the code, we can use the following commands to compile and execute the application:

In [ ]:
!g++ -O3 -march=native -std=c++17 ../src/stencil-2d/stencil-2d-base.cpp -o ../build/stencil-2d-base

In [ ]:
!../build/stencil-2d-base double 8192 8192 2 256

Next, we introduce OpenMP parallelization to enhance *CPU* performance.
The updated version is available in [stencil-2d-omp-host.cpp](../src/stencil-2d/stencil-2d-omp-host.cpp), and can be compiled and executed using:

In [ ]:
!g++ -O3 -march=native -std=c++17 -fopenmp ../src/stencil-2d/stencil-2d-omp-host.cpp -o ../build/stencil-2d-omp-host

In [ ]:
!../build/stencil-2d-omp-host double 8192 8192 2 256

Depending on parameters set and hardware used, performance gains may vary, not be present at all, or you might even observe a performance degradation.

## First Attempt at GPU Acceleration

To offload computations to the GPU, we extend the code with OpenMP target offloading.
In a first attempt, we limit code changes to the `stencil2d` function.
The updated version is in [stencil-2d-omp-target-v0.cpp](../src/stencil-2d/stencil-2d-omp-target-v0.cpp).

In [ ]:
!nvc++ -O3 -march=native -std=c++17 -mp=gpu -target=gpu ../src/stencil-2d/stencil-2d-omp-target-v0.cpp -o ../build/stencil-2d-omp-target-v0

In [ ]:
!../build/stencil-2d-omp-target-v0 double 8192 8192 2 256

Surprisingly, initial GPU performance is worse than the CPU baseline.
While this outcome might not be surprising to you, especially if you have a background in using OpenMP target offloading, we pretend not to know about the pertaining issues for now.

More importantly, we can now already *evaluate* performance, which can be useful to compare different variants of the same application.
What is missing, however, are the answers to the following questions:
* Could performance be better for this particular hard- and software combination?
* If so, how can we pinpoint what optimizations need to be applied where to raise our performance levels?

# Hardware Characteristics

The bottleneck model, like other performance models, applies to both CPUs and GPUs.
Since this course focuses on GPU performance, we will examine the theoretical limits of key GPU bottlenecks.
Relevant information can be found in official data sheets and white papers 
    ([V100](https://images.nvidia.com/content/volta-architecture/pdf/volta-architecture-whitepaper.pdf),
    [A100](https://images.nvidia.com/aem-dam/en-zz/Solutions/data-center/nvidia-ampere-architecture-whitepaper.pdf),
    [H100](https://resources.nvidia.com/en-us-hopper-architecture/nvidia-h100-tensor-c)),
system documentation, third-party resources (such as [TechPowerUp](https://www.techpowerup.com/gpu-specs/)),
and obtained by running micro-benchmarks (see [below](#TODO)).


|                             |  V100 SXM2 32 GB   | A100 SXM 40 GB \| 80 GB |    H100 SXM 94 GB     |    H100 SXM5 80 GB    |
| --------------------------- | :----------------: | :---------------------: | :-------------------: | :-------------------: |
| Availability                |     Bridges-2      |      NHR@FAU Alex       |     NHR@FAU Helma     |       Bridges-2       |
| CUDA                        |        7.0         |           8.0           |          9.0          |          9.0          |
| #Cores                      | 5120<br/>(80 * 64) |   6912<br/>(108 * 64)   | 16896<br/>(132 * 128) | 16896<br/>(132 * 128) |
| FP32 Performance \[TFLOPS\] |         16         |           19            |          67           |          67           |
| FP64 Performance \[TFLOPS\] |         8          |          9.75           |          33           |          33           |
| FP64:FP32 Ratio             |        1:2         |           1:2           |          1:2          |          1:2          |
| Memory \[GB\]               |         32         |        40 \| 80         |          94           |          80           |
| Bandwidth \[GB/s\]          |        898         |      1555 \| 2039       |         3360          |         3360          |
| L2 Cache \[MB\]             |         6          |           40            |          50           |          50           |
| TDP \[W\]                   |        250         |           400           |          700          |          700          |


It is evident that our first attempt fails to hit any of the major performance limits of our GPU.
As such, there is likely significant potential to improve our application's performance.

We start our optimization by getting a first overview of system-wide behavior next.

# Application Level Profiling

We follow a *top-down approach* to performance analysis, starting with a *whole application* performance overview and then narrowing down to specific *hot spots*.
An initial overview can be obtained using Nsight Systems, using either solely the command line interface, or by complementing the analysis with the provided GUI.

## Nsight Systems CLI

First, we compile and execute out benchmark application to make sure that results are as expected.

In [ ]:
!nvc++ -O3 -march=native -std=c++17 -mp=gpu -target=gpu ../src/stencil-2d/stencil-2d-omp-target-v0.cpp -o ../build/stencil-2d-omp-target-v0
!../build/stencil-2d-omp-target-v0 double 8192 8192 2 256

Next, we profile our binary with `nsys profile`.
The most important command line arguments are:
* `--stats=true`: prints a summary of performance statistics on the command line
* `-o ...`: sets the target output profile file
* `--force-overwrite=true`: replaces the profile file if it already exists (instead of aborting)

A full list of available arguments for the profile mode is available as part of the official [documentation](https://docs.nvidia.com/nsight-systems/UserGuide/index.html#cli-profile-command-switch-options).

In [ ]:
!nsys profile --stats=true -o ../profiles/stencil-2d-omp-target-v0 --force-overwrite=true ../build/stencil-2d-omp-target-v0 double 8192 8192 2 256

The output of the command line is organized in multiple categories.
A possible output for an Nvidia A100 (80 GB) is copied in below (truncated for conciseness and readability):

### Possible Output

```
[5/8] Executing 'cuda_api_sum' stats report

 Time (%)  Total Time (ns)  Num Calls   Avg (ns)    Med (ns)   Min (ns)  Max (ns)  StdDev (ns)          Name        
 --------  ---------------  ---------  ----------  ----------  --------  --------  -----------  --------------------
     53.3      23786192424        516  46097272.1  46176391.0  42338870  51323982    3239367.7  cuMemcpyDtoHAsync_v2
     46.7      20848528540        516  40404125.1  40333410.0  40036399  50412082     501856.0  cuMemcpyHtoDAsync_v2
      0.0         21000576          1  21000576.0  21000576.0  21000576  21000576          0.0  cuMemAllocManaged   
      0.0          8406454        258     32583.2     28026.0     16312     85171      13101.7  cuLaunchKernel      
      0.0          1589240          4    397310.0    433931.0     10652    710726     348214.2  cuMemAlloc_v2       
      0.0          1434229        258      5559.0      5245.5      3477     36904       2436.3  cuStreamSynchronize 
      0.0           600084          1    600084.0    600084.0    600084    600084          0.0  cuMemAllocHost_v2   
      0.0           237136          1    237136.0    237136.0    237136    237136          0.0  cuModuleLoadDataEx  
      0.0             3296          4       824.0       701.5       230      1663        638.4  cuCtxSetCurrent     
      0.0             1753          1      1753.0      1753.0      1753      1753          0.0  cuInit              
```

```
[6/8] Executing 'cuda_gpu_kern_sum' stats report

 Time (%)  Total Time (ns)  Instances  Avg (ns)   Med (ns)   Min (ns)  Max (ns)  StdDev (ns)                     Name                    
 --------  ---------------  ---------  ---------  ---------  --------  --------  -----------  -------------------------------------------
    100.0       1620167222        258  6279717.9  6272194.0   6249988   7635683      86899.5  nvkernel__Z9stencil2dIdEvPKT_PS0_mm_F1L5_14
```

```
[7/8] Executing 'cuda_gpu_mem_time_sum' stats report

 Time (%)  Total Time (ns)  Count   Avg (ns)    Med (ns)   Min (ns)  Max (ns)  StdDev (ns)           Operation          
 --------  ---------------  -----  ----------  ----------  --------  --------  -----------  ----------------------------
     51.4      22075935038    516  42782819.8  42666977.0  42240513  44838104     341715.9  [CUDA memcpy Device-to-Host]
     48.6      20832112509    516  40372311.1  40298753.0  39997449  50354473     499040.6  [CUDA memcpy Host-to-Device]
```

```
[8/8] Executing 'cuda_gpu_mem_size_sum' stats report

 Total (MB)  Count  Avg (MB)  Med (MB)  Min (MB)  Max (MB)  StdDev (MB)           Operation          
 ----------  -----  --------  --------  --------  --------  -----------  ----------------------------
 277025.391    516   536.871   536.871   536.871   536.871        0.000  [CUDA memcpy Device-to-Host]
 277025.391    516   536.871   536.871   536.871   536.871        0.000  [CUDA memcpy Host-to-Device]
```

### Interpretation

Looking at the statistics we can see multiple effects:
* The number of kernel instances matches our expectation ($2 + 256$).
* The number of memory transfers seems to be related to the number of kernel instances.
* Comparing the time spent in GPU synchronization (`cuStreamSynchronize`) and kernel execution time shows a mismatch.
* Comparing aggregated memory transfer and kernel execution times reveals a stark difference.
  * \> Even if the kernel could be accelerated, overall performance will most likely not increase.
  * \> These numbers could be used to approximate a minimum number of iterations at which the memory transfers get amortized (assuming that the transfer times *don't scale with the number of iterations*).
* The size per transfer matches our expectation ($8192^2 \cdot 8 B \approx 537 MB$).

## Nsight Systems GUI

Next, we take a closer look at the problematic memory transfers by opening the generated `stencil-2d-omp-target-v0.nsys-rep` report file located in the `profiles` folder.

You can download it directly from the folder, or **shift + right-click** on this [link](../profiles/stencil-2d-omp-target-v0.nsys-rep) and select "Download".
Afterwards, open it up in your local installation of Nsight Systems.

As you maybe already suspected, the timeline shows a recurring pattern of
* two memory transfers (HtoD),
* a kernel call, and
* two memory transfers (DtoH)

Additionally, `cudaStreamSynchronize` is only called at the end of this pattern, which partly explains the deviation from the kernel execution time.

At this stage, we have two options:
* Update our bottleneck model to explicitly include PCIe transfers
* Optimize our application to eliminate unnecessary data transfers

If we choose the first option, we need to consider the peak bandwidth for the PCIe connection, which depends on the GPU generation:


|                                    |  Volta  | Ampere  | Hopper  | 
|------------------------------------|:-------:|:-------:|:-------:|
| PCIe                               | 3.0 x16 | 4.0 x16 | 5.0 x16 |
| Bandwidth (per direction) \[GB/s\] |  15.8   |  31.5   |  63.0   |

For this tutorial, however, we will proceed with the second option and focus on reducing redundant transfers.

## Stencil Code Optimization - Data Transfers

Having pinpointed our performance bug, we can now optimize data transfers in our application.
One straight-forward way is adding unstructured data primitives in our code, basically spanning a region at whose begin and end data is copied *only one time*.
The updated version is available at [stencil-2d-omp-target-v1.cpp](../src/stencil-2d/stencil-2d-omp-target-v1.cpp), and can be compiled, executed and profiled using the following cells.

In [ ]:
!nvc++ -O3 -march=native -std=c++17 -mp=gpu -target=gpu ../src/stencil-2d/stencil-2d-omp-target-v1.cpp -o ../build/stencil-2d-omp-target-v1

In [ ]:
!../build/stencil-2d-omp-target-v1 double 8192 8192 2 256

In [ ]:
!nsys profile --stats=true -o ../profiles/stencil-2d-omp-target-v1 --force-overwrite=true ../build/stencil-2d-omp-target-v1 double 8192 8192 2 256

### Possible Output

```
[5/8] Executing 'cuda_api_sum' stats report

 Time (%)  Total Time (ns)  Num Calls   Avg (ns)    Med (ns)   Min (ns)  Max (ns)  StdDev (ns)          Name        
 --------  ---------------  ---------  ----------  ----------  --------  --------  -----------  --------------------
     89.5       1645737125        260   6329758.2   6268100.0      3938   7623351     631560.0  cuStreamSynchronize 
      4.8         87537361          2  43768680.5  43768680.5  43395807  44141554     527322.8  cuMemcpyDtoHAsync_v2
      4.4         81393811          2  40696905.5  40696905.5  40278531  41115280     591670.9  cuMemcpyHtoDAsync_v2
      1.2         21206432          1  21206432.0  21206432.0  21206432  21206432          0.0  cuMemAllocManaged   
      0.1          1584641          4    396160.3    421646.5     13778    727570     356926.3  cuMemAlloc_v2       
      0.1          1361529        258      5277.2      4654.5      4098     43227       3040.1  cuLaunchKernel      
      0.0           573652          1    573652.0    573652.0    573652    573652          0.0  cuMemAllocHost_v2   
      0.0           236775          1    236775.0    236775.0    236775    236775          0.0  cuModuleLoadDataEx  
      0.0             2776          4       694.0       681.5       411      1002        261.0  cuCtxSetCurrent     
      0.0             1844          1      1844.0      1844.0      1844      1844          0.0  cuInit              
```

```
[6/8] Executing 'cuda_gpu_kern_sum' stats report

 Time (%)  Total Time (ns)  Instances  Avg (ns)   Med (ns)   Min (ns)  Max (ns)  StdDev (ns)                     Name                    
 --------  ---------------  ---------  ---------  ---------  --------  --------  -----------  -------------------------------------------
    100.0       1644696793        258  6374793.8  6264350.0   6244318   7624708     298503.9  nvkernel__Z9stencil2dIdEvPKT_PS0_mm_F1L5_14
```

```
[7/8] Executing 'cuda_gpu_mem_time_sum' stats report

 Time (%)  Total Time (ns)  Count   Avg (ns)    Med (ns)   Min (ns)  Max (ns)  StdDev (ns)           Operation          
 --------  ---------------  -----  ----------  ----------  --------  --------  -----------  ----------------------------
     51.7         86989155      2  43494577.5  43494577.5  43004303  43984852     693352.8  [CUDA memcpy Device-to-Host]
     48.3         81288901      2  40644450.5  40644450.5  40230464  41058437     585465.3  [CUDA memcpy Host-to-Device]
```

```
[8/8] Executing 'cuda_gpu_mem_size_sum' stats report

 Total (MB)  Count  Avg (MB)  Med (MB)  Min (MB)  Max (MB)  StdDev (MB)           Operation          
 ----------  -----  --------  --------  --------  --------  -----------  ----------------------------
   1073.742      2   536.871   536.871   536.871   536.871        0.000  [CUDA memcpy Device-to-Host]
   1073.742      2   536.871   536.871   536.871   536.871        0.000  [CUDA memcpy Host-to-Device]
```

### Interpretation

Looking at the statistics we can now compare the effects previously discussed:
* The number of kernel instances matches our expectation ($2 + 256$).
* ~~The number of memory transfers seems to be related to the number of kernel instances.~~
* ~~Comparing the time spent in GPU synchronization (`cuStreamSynchronize`) and kernel execution time shows a mismatch.~~
* ~~Comparing aggregated memory transfer and kernel execution times reveals an order of magnitude in difference.~~
* The size per transfer matches our expectation ($8192 \cdot 8192 \cdot 8 \text{B} \approx 537 \text{MB}$).

Opening up the generated profile output in our GUI reveals what we already expected: single staging parts at the beginning and the end of our application with multiple kernel calls in between.
The output file is once again collected in the `../profiles` folder and accessible via [link](../profiles/stencil-2d-omp-target-v1.nsys-rep).

# Kernel Level Profiling

After analyzing the application-level view and addressing the most pressing issues, we can now focus on identifying which kernels contribute most to the overall execution time.

In our case, this step is straightforward: the application contains only a single kernel.

We profile the kernel performance of our application using the CLI of Nsight Compute: `ncu`.
Key command line arguments are (all command line arguments are listed in the [documentation](https://docs.nvidia.com/nsight-compute/NsightComputeCli/index.html#profile)):
* `-o ...`: sets the target output profile file (equivalent to `nsys`)
* `--force-overwrite`: replaces the profile file if it already exists (in contrast to `nsys` no `=true`)

By default *every kernel* is profiled.
Apart from adapting the number of iterations in our applications, we can further limit the scope of profiled kernels with
* `--launch-skip n` or `-s n`: skips the first `n` kernels encountered
* `--launch-count n` or `-c n`: limits profiling to the first `n` applicable kernels
* `--kernel name` or `-k name`: limits profiling to kernels with the name `name`
  * can also be used with regex, e.g. `-k regex:"stencil[1-3]d"`
  * Nsight Compute also supports kernel renaming

In [ ]:
!ncu -s 2 -c 1 -o ../profiles/stencil-2d-omp-target-v1 --force-overwrite ../build/stencil-2d-omp-target-v1 double 8192 8192 2 2

If no output file is specified, key profiling insights will be displayed directly in the command line output.

In [ ]:
!ncu -s 2 -c 1 ../build/stencil-2d-omp-target-v1 double 8192 8192 2 2

### Potential Output

```
  nvkernel__Z9stencil2dIdEvPKT_PS0_mm_F1L5_14 (64, 1, 1)x(128, 1, 1), Context 1, Stream 13, Device 0, CC 8.0
    Section: GPU Speed Of Light Throughput
    ----------------------- ----------- ------------
    Metric Name             Metric Unit Metric Value
    ----------------------- ----------- ------------
    DRAM Frequency                  Ghz         1.59
    SM Frequency                    Ghz         1.15
    Elapsed Cycles                cycle      8790515
    Memory Throughput                 %        35.44
    DRAM Throughput                   %         6.99
    Duration                         ms         7.61
    L1/TEX Cache Throughput           %        61.19
    L2 Cache Throughput               %        23.43
    SM Active Cycles              cycle   5089551.15
    Compute (SM) Throughput           %         1.10
    ----------------------- ----------- ------------

    OPT   This kernel grid is too small to fill the available resources on this device, resulting in only 0.0 full      
          waves across all SMs. Look at Launch Statistics for more details.
```

```
   Section: Launch Statistics
    -------------------------------- --------------- ---------------
    Metric Name                          Metric Unit    Metric Value
    -------------------------------- --------------- ---------------
    Block Size                                                   128
    Function Cache Configuration                     CachePreferNone
    Grid Size                                                     64
    Registers Per Thread             register/thread              40
    Shared Memory Configuration Size           Kbyte           32.77
    Driver Shared Memory Per Block       Kbyte/block            1.02
    Dynamic Shared Memory Per Block       byte/block               0
    Static Shared Memory Per Block        byte/block               0
    # SMs                                         SM             108
    Stack Size                                                  1024
    Threads                                   thread            8192
    # TPCs                                                        54
    Enabled TPC IDs                                              all
    Uses Green Context                                             0
    Waves Per SM                                                0.05
    -------------------------------- --------------- ---------------

    OPT   Est. Speedup: 40.74%                                                                                          
          The grid for this launch is configured to execute only 64 blocks, which is less than the GPU's 108            
          multiprocessors. This can underutilize some multiprocessors. If you do not intend to execute this kernel      
          concurrently with other workloads, consider reducing the block size to have at least one block per            
          multiprocessor or increase the size of the grid to fully utilize the available hardware resources. See the    
          Hardware Model (https://docs.nvidia.com/nsight-compute/ProfilingGuide/index.html#metrics-hw-model)            
          description for more details on launch configurations.                                                        
```

```
    Section: Occupancy
    ------------------------------- ----------- ------------
    Metric Name                     Metric Unit Metric Value
    ------------------------------- ----------- ------------
    Block Limit SM                        block           32
    Block Limit Registers                 block           12
    Block Limit Shared Mem                block           32
    Block Limit Warps                     block           16
    Theoretical Active Warps per SM        warp           48
    Theoretical Occupancy                     %           75
    Achieved Occupancy                        %         6.25
    Achieved Active Warps Per SM           warp         4.00
    ------------------------------- ----------- ------------

    OPT   Est. Local Speedup: 91.67%                                                                                    
          The difference between calculated theoretical (75.0%) and measured achieved occupancy (6.2%) can be the       
          result of warp scheduling overheads or workload imbalances during the kernel execution. Load imbalances can   
          occur between warps within a block as well as across blocks of the same kernel. See the CUDA Best Practices   
          Guide (https://docs.nvidia.com/cuda/cuda-c-best-practices-guide/index.html#occupancy) for more details on     
          optimizing occupancy.                                                                                         
    ----- --------------------------------------------------------------------------------------------------------------
    OPT   Est. Local Speedup: 25%                                                                                       
          The 12.00 theoretical warps per scheduler this kernel can issue according to its occupancy are below the      
          hardware maximum of 16. This kernel's theoretical occupancy (75.0%) is limited by the number of required      
          registers.                                                                                                    
```

```
    Section: GPU and Memory Workload Distribution
    -------------------------- ----------- ------------
    Metric Name                Metric Unit Metric Value
    -------------------------- ----------- ------------
    Average DRAM Active Cycles       cycle    847195.70
    Total DRAM Elapsed Cycles        cycle    484965248
    Average L1 Active Cycles         cycle   5089551.15
    Total L1 Elapsed Cycles          cycle    949080162
    Average L2 Active Cycles         cycle   8317935.46
    Total L2 Elapsed Cycles          cycle    666683360
    Average SM Active Cycles         cycle   5089551.15
    Total SM Elapsed Cycles          cycle    949080162
    Average SMSP Active Cycles       cycle   5089300.81
    Total SMSP Elapsed Cycles        cycle   3796320648
    -------------------------- ----------- ------------

    OPT   Est. Speedup: 24.38%                                                                                          
          One or more SMs have a much lower number of active cycles than the average number of active cycles. Maximum   
          instance value is 42.09% above the average, while the minimum instance value is 100.00% below the average.    
    ----- --------------------------------------------------------------------------------------------------------------
    OPT   Est. Speedup: 24.38%                                                                                          
          One or more SMSPs have a much lower number of active cycles than the average number of active cycles. Maximum 
          instance value is 42.09% above the average, while the minimum instance value is 100.00% below the average.    
    ----- --------------------------------------------------------------------------------------------------------------
    OPT   Est. Speedup: 24.38%                                                                                          
          One or more L1 Slices have a much lower number of active cycles than the average number of active cycles.     
          Maximum instance value is 42.09% above the average, while the minimum instance value is 100.00% below the     
          average.

The first section, *GPU Speed Of Light Throughput*, provides a summary of key metrics and potential bottlenecks.
The most relevant for our analysis are:
* `Duration` of a single kernel execution (in ms)
* `DRAM Throughput` as a percentage of the theoretical maximum
* `Compute (SM) Throughput` as a percentage of the theoretical maximum

By default, Nsight Compute also offers suggestions for improving performance.
For example, we might see output such as:
```
OPT   This kernel grid is too small to fill the available resources on this device,
      resulting in only 0.1 full waves across all SMs. 
      Look at Launch Statistics for more details.
```

Depending on your background, this may prompt the question: what is a *wave*?

# GPU Architecture 101

## Example: NVIDIA H100

<img align="right" src="img/h100-chip.png" alt="H100 Chip" width="384px"/>

A fundamental design principle of modern GPUs is their hierarchical structure, which enables high levels of parallelism and scalability.
To better understand GPU performance, we will examine the NVIDIA H100 architecture as a representative example of state-of-the-art GPUs.

Further details can be found in the official [NVIDIA H100 White Paper](https://resources.nvidia.com/en-us-tensor-core).
The following figures and information are also derived from this resource.

<img align="right" src="img/h100-layout.png" alt="H100 Chip Layout with Annotations" width="576px" style="background-color:white"/>

The diagram on the right depicts the 'full configuration' of a single NVIDIA H100 chip.
However, not all functional units shown are accessible in production GPUs.
For example, the SXM5 version of the H100 features **132 Streaming Multiprocessors (SMs)** enabled for computation.

<img align="right" src="img/h100-sm-layout.png" alt="H100 SM Layout" width="384px" style="background-color:white"/>

Each SM is subdivided into 4 sub partitions (SMSP), as visualized in the figure on the right, each with
* 16 INT32 units, for a total of $132 \cdot 4 \cdot 16 = 8448$,
* 32 FP32 units, for a total of $132 \cdot 4 \cdot 32 = 16896$,
* 16 FP64 units, for a total of $132 \cdot 4 \cdot 16 = 8448$, and
* 1 tensor core, for a total of $132 \cdot 4 = 528$.

Each unit is capable of executing one fused-multiply-add (FMA) operation per cycle, with the exception of the tensor cores which can each perform 512 FP16/FP32-mixed-precision FMAs at once.

## Memory Hierarchy

<img align="right" src="img/h100-memory-abstract.png" alt="memory abstraction" width="512px"/>

The GPU memory system is organized into a hierarchy designed to balance capacity, speed, and access latency.
Its levels, at the example of the H100, are (latency and bandwidth values provided by [GPU Benches](https://github.com/RRZE-HPC/gpu-benches))
- **Registers**
    - Scope: Private to each thread.
    - Latency: Typically 1 cycle.
    - Capacity: Each SM has 256 KB available for registers.
- **L1 Cache** and **Shared Memory**
    - Scope: Shared among all threads scheduled on a given SM. Threads of a given block can access the same shared memory.
    - Latency: ~32 cycles.
    - Capacity: Configurable up to 100 KB per SM for shared memory, with the remaining space used for L1 cache (up to 256 KB per SM).
- **L2 Cache**
    - Scope: Shared among all threads (GPU-wide)
    - Latency: ~280 cycles.
    - Capacity: 50 MB (two partitions with 25 MB each)
- **Global Memory (DRAM)**
    - Scope: Global to all threads.
    - Latency: ~690 cycles.
    - Capacity: 80 or 96 GB
    - Bandwidth: up to 3.35 TB/s.

## Waves and Occupancy

While this detour into GPU architecture was interesting, we still have an open question: what is a *wave*?

To answer this, let's clarify how threads are organized on GPUs:
- **OpenMP** organizes *threads* in *teams*.
- **CUDA** organizes *threads* in *blocks*.

Although OpenMP teams and CUDA thread blocks are not strictly equivalent, compilers typically map teams to blocks when targeting NVIDIA GPUs.
These blocks are then assigned to Streaming Multiprocessors (SMs):
* Each block is scheduled to one SM.
* Each SM can handle multiple blocks concurrently (depending on available resources).

Within a block, threads are further grouped into *warps*: groups of 32 threads.
Warps are the basic unit of scheduling on the SM sub-partitions (SMSPs).

A *wave* refers to the maximum number of blocks that can be resident on the GPU at the same time.
It is calculated as the number of SMs times the maximum number of blocks per SM.

Having at least one full wave ensures that all resources are fully utilized.
Multiple (full) waves work as well, but partial waves may lead to sub-optimal performance.

This closely relates to the concept of *occupancy* which is defined as the ratio of active warps to the maximum number of warps that can be scheduled on an SM.

# Micro Benchmarks

At this point, you might wonder whether achieving full waves is always necessary, or if a lower degree of parallelism can sometimes suffice.

There are two main ways to approach this question:

1. **Conceptual Analysis:** \
To have a GPU fully utilize its computational resources, it requires at least as many active threads as there are execution units for the current data type - typically on the order of ten thousand.
However, most applications require additional data transfers between main memory (or at least L2 cache) and the compute units, which introduces high latency.
To hide this latency and keep the GPU busy, resources need to be oversubscribed: when some warps are stalled waiting for data, others can be scheduled to execute.
In practice this usually relates to an order of magnitude more threads required (on the order of a hundred thousand).

2. **Benchmarking:** \
(Micro) benchmarks can help study performance for varying number of threads.
They can additionally be used to examine the effects of different data types, memory access patterns, computational intensities, the ratio of compute to data transfer, and more.\
Let's look at two practical examples from the [Accelerated Programming EXamples (APEX)](https://github.com/SebastianKuckuk/apex) collection: a [fused-multiply-add (FMA)](https://github.com/SebastianKuckuk/apex/tree/main/src/benchmark/fma) benchmark to assess computational throughput, and a [copy-stream](https://github.com/SebastianKuckuk/apex/tree/main/src/benchmark/stream) benchmark to evaluate memory bandwidth and data transfer efficiency.
Automated benchmarking of both is available as part of the [APEX-Generator](https://github.com/SebastianKuckuk/apex-generator) project. 

## FMA Benchmark

<img align="right" src="img/A100-SXM4-80GB---fma.png" alt="FMA Benchmark Results for A100 80GB" width="600px"/>

On the right, we can see the results for the FMA benchmark on a single A100 80GB and the following effects:
* The maximum performance observed is $9.7$ PFLOP/s.
    * This is pretty much the theoretical limit given by the data sheet for double precision.
* A distinct saw-tooth pattern is visible.
    * The first peak is at $26\,615$ threads, the first drop is at $28\,526$ threads. 
    * This is just around $256 \cdot 108 = 27\,648$ threads, i.e. when each SM gets exactly one thread block.
    * At the first drop, performance nearly halves because the amount of work increases only slightly, but the execution time doubles due to one SM having doubled workload. 
* Compared to the CUDA version, the compiler seems to adapt block sizes when using OpenMP target offloading. 

## Stream Benchmark

<img align="right" src="img/A100-SXM4-80GB---stream.png" alt="Stream Benchmark Results for A100 80GB" width="600px"/>

On the right, we can see the results for the stream benchmark on a single A100 80GB and the following effects:
* The asymptotic bandwidth is around $1\,750$ GB/s, which is just around $85\%$ of the theoretical limit of $2\,040$ GB/s.
* The maximum bandwidth is around $2\,580$ GB/s for $1\,290\,948$ threads.
    * Each thread moves 16 bytes in the double precision case.
    * This corresponds to a total volume of just over $20$ MB, one L2 cache partition on the A100.

## Stencil Code Optimization - Collapse

As we have seen in the last section about micro benchmarking, we ideally have hundreds of thousands of threads.
Revising the code in [stencil-2d-omp-target-v1](../src/stencil-2d/stencil-2d-omp-target-v1.cpp) shows that the current degree of parallelism is limited by the number of iterations of *the outer loop*, in our case 8190.

To increase the number of threads we need to generate more parallelism.
Luckily, simply associating more loops with our OpenMP construct serves exactly that purpose.
One simple way of achieving this is using the `collapse` clause.
The updated code is available at [stencil-2d-omp-target-v2](../src/stencil-2d/stencil-2d-omp-target-v2.cpp), and can be compiled, executed and profiled with the following cells.

In [ ]:
!nvc++ -O3 -march=native -std=c++17 -mp=gpu -target=gpu ../src/stencil-2d/stencil-2d-omp-target-v2.cpp -o ../build/stencil-2d-omp-target-v2

In [ ]:
!../build/stencil-2d-omp-target-v2 double 8192 8192 2 256

In [ ]:
!ncu -s 2 -c 1 ../build/stencil-2d-omp-target-v2 double 8192 8192 2 2

### Potential Output

```
  nvkernel__Z9stencil2dIdEvPKT_PS0_mm_F1L5_14 (524033, 1, 1)x(128, 1, 1), Context 1, Stream 13, Device 0, CC 8.0
    Section: GPU Speed Of Light Throughput
    ----------------------- ----------- ------------
    Metric Name             Metric Unit Metric Value
    ----------------------- ----------- ------------
    DRAM Frequency                  Ghz         1.59
    SM Frequency                    Ghz         1.15
    Elapsed Cycles                cycle      1058644
    Memory Throughput                 %        56.95
    DRAM Throughput                   %        56.95
    Duration                         us       916.77
    L1/TEX Cache Throughput           %        27.80
    L2 Cache Throughput               %        81.41
    SM Active Cycles              cycle   1055577.11
    Compute (SM) Throughput           %        64.20
    ----------------------- ----------- ------------

    INF   Compute and Memory are well-balanced: To reduce runtime, both computation and memory traffic must be reduced. 
          Check both the Compute Workload Analysis and Memory Workload Analysis sections.                               
```

That already looks a lot better.
The next question we might ask is: how accurate are our estimated bandwidth numbers?

# Profiling Metrics

Fortunately, Nsight Compute can also be used to get accurately *measure* bandwidth values for single kernel executions.
While the more manual approach demonstrated next might not be particularly useful for general performance analysis and engineering, it can be quite useful for automated measurements of isolated performance metrics.
It also helps in keeping profiling overheads low.

Nvidia publishes additional material on the [metrics structure](https://docs.nvidia.com/nsight-compute/ProfilingGuide/index.html#metrics-structure), as well as a [list of key metrics](https://docs.nvidia.com/nsight-compute/NsightComputeCli/index.html#nvprof-metric-comparison).
Additionally, Nsight Compute can list available metrics (in a very long list).

In [ ]:
!ncu --list-metrics > ../profiles/metrics.txt

In [ ]:
!cat ../profiles/metrics.txt

Since the list is quite long and not particularly easy to navigate, let's review some relevant metrics:
* `sm__warps_active.avg.pct_of_peak_sustained_active` displays the achieved occupancy.
* `dram__bytes_read` and `dram__bytes_write` correspond to the bytes read from and written to DRAM.
    This can be extended with `.sum` to obtain the total volume and `.sum.per_second` to obtain the bandwidth.
* `smsp__sass_thread_inst_executed_op_{dadd, dmul, dfma}_pred_on.sum` represents the total executed additions, multiplications and fused multiply-adds in double precision.
    The total number of FLOPs in double precision can be computed with `dadd + dmul + 2 * dfma`.
* `lts__t_bytes_equiv_l1sectormiss_pipe_lsu_mem_global_op_{ld, st}.sum` can be used to query L2 load and store volumes.

Metrics to be measured can be provided on the command line using the optional argument `--metrics`:

In [ ]:
!ncu -s 2 -c 1 --metrics sm__warps_active.avg.pct_of_peak_sustained_active,dram__bytes_read.sum,dram__bytes_write.sum,dram__bytes_read.sum.per_second,dram__bytes_write.sum.per_second,smsp__sass_thread_inst_executed_op_dadd_pred_on.sum,smsp__sass_thread_inst_executed_op_dmul_pred_on.sum,smsp__sass_thread_inst_executed_op_dfma_pred_on.sum,smsp__sass_thread_inst_executed_op_dadd_pred_on.sum.per_second,smsp__sass_thread_inst_executed_op_dmul_pred_on.sum.per_second,smsp__sass_thread_inst_executed_op_dfma_pred_on.sum.per_second ../build/stencil-2d-omp-target-v2 double 8192 8192 2 2

Some general guidelines:
* For counting metrics (e.g. bytes transferred) the structure is usually `metric.sum`.
* For corresponding rates this can be extended to `metric.sum.per_second`.
* Many metrics can be recomputed in terms of percentage of theoretical peak performance with `metric.sum.pct_of_peak_sustained_elapsed`.

## Further Nsight Compute Options

Providing a custom set of metrics can be quite useful, but also extremely cumbersome.
A more coarse-grained way of telling Nsight Compute what to measure is using **sections** or **sets**.

### Sections

The available sections can be queried with

In [ ]:
!ncu --list-sections > ../profiles/sections.txt

In [ ]:
!cat ../profiles/sections.txt

and used with the `--section` parameter

In [ ]:
!ncu -s 2 -c 1 --section=SpeedOfLight ../build/stencil-2d-omp-target-v2 double 8192 8192 2 2

### Sets

In many cases, multiple sections are required concurrently.
Sets provide an interface for the most relevant combinations.
As before, they can be queried directly from `ncu` and then used with the corresponding command line argument.

In [ ]:
!ncu --list-sets > ../profiles/sets.txt

In [ ]:
!cat ../profiles/sets.txt

In [ ]:
!ncu -s 2 -c 1 --set=roofline ../build/stencil-2d-omp-target-v2 double 8192 8192 2 2

# Nsight Compute GUI

While the command line interface for Nsight Compute is quite powerful, in many cases a more structured and explorable way of presenting the obtained data is advantageous.
In this case, the Nsight Compute GUI can be a helpful tool.
We follow this pattern:
* Profiling of our application with suitable sections/ sets remotely.
* Downloading the profile data.
* Opening it locally.

In [ ]:
!nvc++ -O3 -march=native -std=c++17 -mp=gpu -target=gpu ../src/stencil-2d/stencil-2d-omp-target-v2.cpp -o ../build/stencil-2d-omp-target-v2

In [ ]:
!../build/stencil-2d-omp-target-v2 double 8192 8192 2 256

In [ ]:
!ncu --set=full -o ../profiles/stencil-2d-omp-target-v2 --force-overwrite ../build/stencil-2d-omp-target-v2 double 8192 8192 2 2

As before, the obtained profile file `stencil-2d-omp-target-v2.ncu-rep` can be downloaded directly from the `profiles` folder, or by **shift + right-clicking** this [link](../profiles/stencil-2d-omp-target-v2.ncu-rep).
Afterwards, open it up with your local installation of Nsight Compute.

The volume of information can be quite overwhelming.
Start by double-clicking on (one of) the displayed kernels.
Take a moment to explore the different sections.
You can expand them by clicking the little triangle next to them.
If you are unsure about certain terms you can check short documentation snippets in the form of tooltips by hovering over what you want to know more about.

Let's focus on the **Memory Workload Analysis** section first.
It reveals that the data volume read from the L2 cache is quite high compared to the one from DRAM (about a factor of 3x).
To understand why this is happening, we first have to remember the thread distribution employed by OpenMP.

The resulting access pattern leads to about three times as much data being read from L2, which fits the measured values.
To remedy this behavior, using (spatial) blocking techniques can be one option.
Another is switching to CUDA which naturally allows having a 2D thread decomposition.

# Stencil Code Optimization - Spatial Blocking

[stencil-2d-cuda-v3](../src/stencil-2d/stencil-2d-cuda-v3.cu) implements the approach detailed above.
As ususal, the next cells can be used to compile and execute it.
Also note the switch from `nvc++` to `nvcc`.

In [ ]:
!nvcc -O3 -std=c++17 ../src/stencil-2d/stencil-2d-cuda-v3.cu -o ../build/stencil-2d-cuda-v3

In [ ]:
!../build/stencil-2d-cuda-v3 double 8192 8192 2 256

Using our CUDA implementation seems to have reduced performance again.
For now, we pretend we don't know what is going on, even if you already spotted the (performance) bug.

We start by checking for the same issue as before - maybe there are additional memory transfers again?

In [ ]:
!nsys profile --stats=true -o ../profiles/stencil-2d-cuda-v3 --force-overwrite=true ../build/stencil-2d-cuda-v3 double 8192 8192 2 256

### Potential Output

```
[5/8] Executing 'cuda_api_sum' stats report

 Time (%)  Total Time (ns)  Num Calls    Avg (ns)      Med (ns)    Min (ns)   Max (ns)    StdDev (ns)            Name         
 --------  ---------------  ---------  ------------  ------------  --------  -----------  ------------  ----------------------
     95.8      12198873816          2  6099436908.0  6099436908.0  95515042  12103358774  8490827730.3  cudaDeviceSynchronize 
      3.0        377821142          2   188910571.0   188910571.0  96573958    281247184   130583690.4  cudaMallocHost        
      0.6         81727359          4    20431839.8    20350178.5  20163428     20863574      312054.0  cudaMemcpy            
      0.6         70621881          2    35310940.5    35310940.5  34722770     35899111      831798.7  cudaFreeHost          
      0.0          3533005        258       13693.8        2560.0      2425      2830601      176058.3  cudaLaunchKernel      
      0.0          1442164          2      721082.0      721082.0    720767       721397         445.5  cudaFree              
      0.0          1143135          2      571567.5      571567.5    322668       820467      351997.0  cudaMalloc            
      0.0              772          1         772.0         772.0       772          772           0.0  cuModuleGetLoadingMode
```

```
[6/8] Executing 'cuda_gpu_kern_sum' stats report

 Time (%)  Total Time (ns)  Instances   Avg (ns)    Med (ns)   Min (ns)  Max (ns)  StdDev (ns)                                   Name                                 
 --------  ---------------  ---------  ----------  ----------  --------  --------  -----------  ----------------------------------------------------------------------
    100.0      12199314275        258  47284163.9  47305954.0  46507675  49070476     289519.5  void stencil2d<double>(const T1 *, T1 *, unsigned long, unsigned long)
```

```
[7/8] Executing 'cuda_gpu_mem_time_sum' stats report

 Time (%)  Total Time (ns)  Count   Avg (ns)    Med (ns)   Min (ns)  Max (ns)  StdDev (ns)           Operation          
 --------  ---------------  -----  ----------  ----------  --------  --------  -----------  ----------------------------
     50.6         41222752      2  20611376.0  20611376.0  20434207  20788545     250554.8  [CUDA memcpy Device-to-Host]
     49.4         40298038      2  20149019.0  20149019.0  20146011  20152027       4254.0  [CUDA memcpy Host-to-Device]
```

```
[8/8] Executing 'cuda_gpu_mem_size_sum' stats report

 Total (MB)  Count  Avg (MB)  Med (MB)  Min (MB)  Max (MB)  StdDev (MB)           Operation          
 ----------  -----  --------  --------  --------  --------  -----------  ----------------------------
   1073.742      2   536.871   536.871   536.871   536.871        0.000  [CUDA memcpy Device-to-Host]
   1073.742      2   536.871   536.871   536.871   536.871        0.000  [CUDA memcpy Host-to-Device]
```

Nsight Systems reveals that there are no spurious transfers and that the majority of the time is indeed spent in executing the kernel(s).
Next, we create a profile with Nsight Compute.

In [ ]:
!ncu --set=full -o ../profiles/stencil-2d-cuda-v3 --force-overwrite ../build/stencil-2d-cuda-v3 double 8192 8192 2 2

After downloading the [report file](../profiles/stencil-2d-cuda-v3.ncu-rep) and opening it locally, we compare this profile with the last OpenMP one.
To do so, we can click on `Compare` at the top right and then on `Add Baseline`.
After opening the profile for the CUDA version, we once again focus on the memory workload section.
We can see multiple effects:
* The data volume read from L2 is now a lot lower (about 2x).
* The data volume read from and written to DRAM is now about zero.
* There is a new data path to system memory.

# Final Stencil Code Optimization

The previous analysis reports data transfers from and to system memory, or in other words the host memory, instead of using the GPU memory.
Reviewing the code reveals the culprit: instead of device pointers, we pass host pointers to the kernel.
That is an easy fix - an updated code version is available at [stencil-2d-cuda-v4](../src/stencil-2d/stencil-2d-cuda-v4.cu).

In [ ]:
!nvcc -O3 -std=c++17 ../src/stencil-2d/stencil-2d-cuda-v4.cu -o ../build/stencil-2d-cuda-v4

In [ ]:
!../build/stencil-2d-cuda-v4 double 8192 8192 2 256

The performance of our benchmark application is now quite close to the theoretical limit.
Further optimization of the implementation will likely not bring any performance benefits.
To further accelerate the application, *algorithmic optimizations* are required, e.g. temporal blocking or switching to a different numerical solver algorithm.

Nsight compute also shows traces of this when checking the roofline section (download the profile [here](../profiles/stencil-2d-cuda-v4.ncu-rep)) after running the next cell.
You can review it in the **GPU Speed of Light Throughput** section by choosing **Roofline Double Precision** in the top right drop-down.

In [ ]:
!ncu --set=full -o ../profiles/stencil-2d-cuda-v4 --force-overwrite ../build/stencil-2d-cuda-v4 double 8192 8192 2 2

Roofline models are an extended version of the discussed bottleneck model.
Further details are beyond the scope of this introduction but, generally speaking: points in the chart that are close to the roofline indicate that the profiled code utilizes the hardware well.

# Next Steps

**Common Patterns**

In cases where performance is not close to theoretical limits there are often common patterns to be observed that provide and explanation:
* **Thread divergence**: threads of the same warp take different code paths.
* **(Un-)coalesced memory accesses**: consecutive threads access (non-)consecutive memory locations.
* **Kernel launch overhead**: many short-running kernels lead to a low ratio of time spent in kernels over total execution time.
* **Low occupancy**: lack of parallelism, and/ or overusing available resources lead to a low occupancy.

**Additional Material**

* Many computing centers (regional, national and beyond) around the world offer excellent trainings, often online and free of charge for members of academia.
* [GTC](https://www.nvidia.com/gtc/) offers a wide spectrum of material around GPU computing. There is also an [on-demand archive](https://www.nvidia.com/on-demand/) with thousands of videos.